* Radif Ramadan

* Data Science / JCSDOL-014

* Modul 3 - Day 42

---
---

In [91]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 30)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

### **Gunakan melb_data.csv (Melbourne House Data) dengan:**
- Features: `'Suburb'`, `'Rooms'`, `'Type'`, `'Method'`, `'Distance'`, `'Bedroom2'`, `'Bathroom'`, `'Car'`, `'Landsize'`, `'BuildingArea'`, `'YearBuilt'`, `'CouncilArea'`, `'Regionname' `
- Target: `'Price'`

In [92]:
df_melb = pd.read_csv('melb_data.csv')
df_melb.info()
df_melb.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10199 entries, 0 to 10198
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         10199 non-null  object 
 1   Address        10199 non-null  object 
 2   Rooms          10199 non-null  int64  
 3   Type           10199 non-null  object 
 4   Price          10199 non-null  float64
 5   Method         10199 non-null  object 
 6   SellerG        10199 non-null  object 
 7   Date           10199 non-null  object 
 8   Distance       10199 non-null  float64
 9   Postcode       10199 non-null  float64
 10  Bedroom2       10199 non-null  float64
 11  Bathroom       10199 non-null  float64
 12  Car            10199 non-null  float64
 13  Landsize       10199 non-null  float64
 14  BuildingArea   5426 non-null   float64
 15  YearBuilt      6260 non-null   float64
 16  CouncilArea    10198 non-null  object 
 17  Lattitude      10198 non-null  float64
 18  Longti

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,"1,480,000.00",S,Biggin,3/12/2016,2.50,"3,067.00",2.00,1.00,1.00,202.00,NaN,NaN,Yarra,-37.80,145.00,Northern Metropolitan,"4,019.00"
1,Abbotsford,25 Bloomburg St,2,h,"1,035,000.00",S,Biggin,4/02/2016,2.50,"3,067.00",2.00,1.00,0.00,156.00,79.00,"1,900.00",Yarra,-37.81,144.99,Northern Metropolitan,"4,019.00"
2,Abbotsford,5 Charles St,3,h,"1,465,000.00",SP,Biggin,4/03/2017,2.50,"3,067.00",3.00,2.00,0.00,134.00,150.00,"1,900.00",Yarra,-37.81,144.99,Northern Metropolitan,"4,019.00"
3,Abbotsford,40 Federation La,3,h,"850,000.00",PI,Biggin,4/03/2017,2.50,"3,067.00",3.00,2.00,1.00,94.00,NaN,NaN,Yarra,-37.80,145.00,Northern Metropolitan,"4,019.00"
4,Abbotsford,55a Park St,4,h,"1,600,000.00",VB,Nelson,4/06/2016,2.50,"3,067.00",3.00,1.00,2.00,120.00,142.00,"2,014.00",Yarra,-37.81,144.99,Northern Metropolitan,"4,019.00"


In [93]:
x = df_melb[['Suburb', 'Rooms', 'Type', 'Method', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 
             'BuildingArea', 'YearBuilt', 'CouncilArea', 'Regionname']]
y = df_melb['Price']

### **Lakukan EDA secara sederhana untuk eksplorasi dataset.**

In [94]:
df_melb.describe(include='all')

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
count,10199,10199,"10,199.00",10199,"10,199.00",10199,10199,10199,"10,199.00","10,199.00","10,199.00","10,199.00","10,199.00","10,199.00","5,426.00","6,260.00",10198,"10,198.00","10,198.00",10198,"10,198.00"
unique,275,10060,NaN,3,NaN,5,230,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,NaN,NaN,8,NaN
top,Reservoir,2 Bruce St,NaN,h,NaN,S,Nelson,3/06/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moreland,NaN,NaN,Southern Metropolitan,NaN
freq,286,3,NaN,6770,NaN,6749,1291,395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1043,NaN,NaN,3898,NaN
mean,NaN,NaN,2.87,NaN,"1,084,513.47",NaN,NaN,NaN,9.07,"3,099.11",2.83,1.51,1.55,486.95,144.44,"1,962.76",NaN,-37.81,144.99,NaN,"7,438.83"
std,NaN,NaN,0.96,NaN,"649,263.06",NaN,NaN,NaN,4.58,73.52,0.97,0.69,0.94,"1,302.06",140.24,38.26,NaN,0.07,0.09,NaN,"4,366.11"
min,NaN,NaN,1.00,NaN,"85,000.00",NaN,NaN,NaN,0.00,"3,000.00",0.00,0.00,0.00,0.00,0.00,"1,196.00",NaN,-38.18,144.56,NaN,389.00
25%,NaN,NaN,2.00,NaN,"650,000.00",NaN,NaN,NaN,5.90,"3,044.00",2.00,1.00,1.00,150.00,90.00,"1,940.00",NaN,-37.85,144.93,NaN,"4,392.00"
50%,NaN,NaN,3.00,NaN,"910,000.00",NaN,NaN,NaN,8.80,"3,081.00",3.00,1.00,1.00,370.00,123.00,"1,970.00",NaN,-37.80,145.00,NaN,"6,543.00"
75%,NaN,NaN,3.00,NaN,"1,350,000.00",NaN,NaN,NaN,11.80,"3,146.00",3.00,2.00,2.00,631.00,170.00,"1,999.00",NaN,-37.76,145.05,NaN,"10,331.00"


In [95]:
df_melb.isna().sum()

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                 0
Landsize            0
BuildingArea     4773
YearBuilt        3939
CouncilArea         1
Lattitude           1
Longtitude          1
Regionname          1
Propertycount       1
dtype: int64

### **Splitting dataset dengan proporsi 80:20 dan random_state = 2020**

In [96]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 2020)

### **Gunakan ColumnTransformer untuk imputasi missing value dan encoding.**
- Ada 4 features yang mengandung missing value. Isi missing value tersebut 
dengan imputer yang menurut kamu paling sesuai dengan karakter masing-masing 
feature. 
- Ada 5 features categorical. Lakukan encoding pada features tersebut sesuai 
dengan karakter masing-masing feature. 

In [97]:
num_features = x.select_dtypes(['int', 'float']).columns.to_list()
categorical_features =  x.select_dtypes(['object']).columns.to_list()

- **Feature Numeric akan di imputasi menggunakan Median** --> Hal ini karena Media tahan terhadap Outlier
- **Feature Categorical akan di imputasi menggunakan Most_Frequent (Mode) kemudian akan diubah menggunakan encoding OneHotEncoder** --> Hal ini karena Fitur kategorikal (object) hanya dapat di imputasi menggunakan mode

In [98]:
# Transformer

num_transformer = SimpleImputer(strategy = 'median')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown = 'ignore',sparse_output=False))
])

# Col Transformer

preprocessor = ColumnTransformer(transformers = [
    ('num', num_transformer, num_features),
    ('cat', categorical_transformer, categorical_features)
])

In [99]:
# Fit dan transform data train and test
X_train_transform = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test_transform = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

print("Data Train: ")
display(X_train_transform.head())
print("\nData Test: ")
display(X_test_transform.head())

Data Train: 


,num__Rooms,num__Distance,num__Bedroom2,num__Bathroom,num__Car,num__Landsize,num__BuildingArea,num__YearBuilt,cat__Suburb_Abbotsford,cat__Suburb_Aberfeldie,cat__Suburb_Airport West,cat__Suburb_Albanvale,cat__Suburb_Albert Park,cat__Suburb_Albion,cat__Suburb_Alphington,...,cat__CouncilArea_Stonnington,cat__CouncilArea_Unavailable,cat__CouncilArea_Whitehorse,cat__CouncilArea_Whittlesea,cat__CouncilArea_Wyndham,cat__CouncilArea_Yarra,cat__CouncilArea_Yarra Ranges,cat__Regionname_Eastern Metropolitan,cat__Regionname_Eastern Victoria,cat__Regionname_Northern Metropolitan,cat__Regionname_Northern Victoria,cat__Regionname_South-Eastern Metropolitan,cat__Regionname_Southern Metropolitan,cat__Regionname_Western Metropolitan,cat__Regionname_Western Victoria
0,1.00,2.80,1.00,1.00,0.00,"1,414.00",47.00,"2,014.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
1,4.00,3.50,4.00,3.00,2.00,210.00,210.00,"2,010.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
2,3.00,13.10,3.00,1.00,1.00,590.00,123.00,"1,968.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,3.00,8.40,3.00,1.00,2.00,590.00,94.00,"1,950.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
4,2.00,11.20,2.00,1.00,2.00,363.00,123.00,"1,968.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00



Data Test: 


,num__Rooms,num__Distance,num__Bedroom2,num__Bathroom,num__Car,num__Landsize,num__BuildingArea,num__YearBuilt,cat__Suburb_Abbotsford,cat__Suburb_Aberfeldie,cat__Suburb_Airport West,cat__Suburb_Albanvale,cat__Suburb_Albert Park,cat__Suburb_Albion,cat__Suburb_Alphington,...,cat__CouncilArea_Stonnington,cat__CouncilArea_Unavailable,cat__CouncilArea_Whitehorse,cat__CouncilArea_Whittlesea,cat__CouncilArea_Wyndham,cat__CouncilArea_Yarra,cat__CouncilArea_Yarra Ranges,cat__Regionname_Eastern Metropolitan,cat__Regionname_Eastern Victoria,cat__Regionname_Northern Metropolitan,cat__Regionname_Northern Victoria,cat__Regionname_South-Eastern Metropolitan,cat__Regionname_Southern Metropolitan,cat__Regionname_Western Metropolitan,cat__Regionname_Western Victoria
0,2.00,8.00,2.00,1.00,1.00,201.00,80.00,"1,995.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
1,2.00,21.50,2.00,1.00,1.00,709.00,123.00,"1,968.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
2,2.00,11.20,2.00,1.00,1.00,117.00,62.00,"2,012.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
3,3.00,5.90,3.00,1.00,1.00,555.00,123.00,"1,968.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
4,3.00,5.20,3.00,1.00,1.00,438.00,123.00,"1,968.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00


### **Lakukan Scaling pada numerical features. Pilih scaler yang menurut kamu paling sesuai berdasarkan karakteristik datanya.**

* Scalling Numeric akan dilakukan menggunakan Robust Scaller, karna tahan terhadap outlier

In [100]:
scaler = RobustScaler()

X_train_transform_scaled= scaler.fit_transform(X_train_transform)
X_test_transform_scaled = scaler.transform(X_test_transform)

X_train_transform_scaled = pd.DataFrame(X_train_scaled, columns=scaler.get_feature_names_out())
X_test_transform_scaled= pd.DataFrame(X_test_scaled, columns=scaler.get_feature_names_out())

print("Data Train: ")
display(X_train_transform_scaled.head())
print("\nData Test: ")
display(X_test_transform_scaled.head())

Data Train: 


,num__Rooms,num__Distance,num__Bedroom2,num__Bathroom,num__Car,num__Landsize,num__BuildingArea,num__YearBuilt,cat__Suburb_Abbotsford,cat__Suburb_Aberfeldie,cat__Suburb_Airport West,cat__Suburb_Albanvale,cat__Suburb_Albert Park,cat__Suburb_Albion,cat__Suburb_Alphington,...,cat__CouncilArea_Stonnington,cat__CouncilArea_Unavailable,cat__CouncilArea_Whitehorse,cat__CouncilArea_Whittlesea,cat__CouncilArea_Wyndham,cat__CouncilArea_Yarra,cat__CouncilArea_Yarra Ranges,cat__Regionname_Eastern Metropolitan,cat__Regionname_Eastern Victoria,cat__Regionname_Northern Metropolitan,cat__Regionname_Northern Victoria,cat__Regionname_South-Eastern Metropolitan,cat__Regionname_Southern Metropolitan,cat__Regionname_Western Metropolitan,cat__Regionname_Western Victoria
0,-2.00,-1.02,-2.00,0.00,-1.00,2.14,-9.50,4.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
1,1.00,-0.90,1.00,2.00,1.00,-0.35,10.88,4.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
2,0.00,0.73,0.00,0.00,0.00,0.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,-0.07,0.00,0.00,1.00,0.43,-3.62,-1.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
4,-1.00,0.41,-1.00,0.00,1.00,-0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00



Data Test: 


,num__Rooms,num__Distance,num__Bedroom2,num__Bathroom,num__Car,num__Landsize,num__BuildingArea,num__YearBuilt,cat__Suburb_Abbotsford,cat__Suburb_Aberfeldie,cat__Suburb_Airport West,cat__Suburb_Albanvale,cat__Suburb_Albert Park,cat__Suburb_Albion,cat__Suburb_Alphington,...,cat__CouncilArea_Stonnington,cat__CouncilArea_Unavailable,cat__CouncilArea_Whitehorse,cat__CouncilArea_Whittlesea,cat__CouncilArea_Wyndham,cat__CouncilArea_Yarra,cat__CouncilArea_Yarra Ranges,cat__Regionname_Eastern Metropolitan,cat__Regionname_Eastern Victoria,cat__Regionname_Northern Metropolitan,cat__Regionname_Northern Victoria,cat__Regionname_South-Eastern Metropolitan,cat__Regionname_Southern Metropolitan,cat__Regionname_Western Metropolitan,cat__Regionname_Western Victoria
0,-1.00,-0.14,-1.00,0.00,0.00,-0.37,-5.38,2.70,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
1,-1.00,2.15,-1.00,0.00,0.00,0.68,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
2,-1.00,0.41,-1.00,0.00,0.00,-0.54,-7.62,4.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
3,0.00,-0.49,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
4,0.00,-0.61,0.00,0.00,0.00,0.12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00


Lakukan prediksi dengan metode Lasso pada data preprocessed. 

In [101]:
lasso_model = Lasso()

lasso_model.fit(X_train_transform_scaled, y_train)

y_train_pred = lasso_model.predict(X_train_transform_scaled)
y_test_pred = lasso_model.predict(X_test_transform_scaled)

e:\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.099e+14, tolerance: 3.265e+11
  model = cd_fast.enet_coordinate_descent(


In [102]:
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

mape_train = mean_absolute_percentage_error(y_train, y_train_pred)
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)

evaluation_df = pd.DataFrame({
    "Dataset": ["Train", "Test"],
    "MSE": [mse_train, mse_test],
    "MAPE (%)": [mape_train, mape_test],
    "R² Score": [r2_train, r2_test]
})

pd.options.display.float_format = '{:,.2f}'.format  

evaluation_df


,Dataset,MSE,MAPE (%),R² Score
0,Train,"130,922,134,220.88",0.26,0.69
1,Test,"121,675,079,702.02",0.26,0.70
